In [5]:
import base64
import os

import cv2
import numpy as np
import requests
import pickle

url = 'http://localhost:8080/api/gethog'

class_brand = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce',
               'Swift', 'Tata Safari', 'Toyota Innova']

def img2HOG(img):
    v,buffer = cv2.imencode('.jpg',img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url,json={"img":data})
    
    return response.json()

def readData(path):
    response = []
    for folder in os.listdir(path):
        for img_name in os.listdir(path+'/'+folder):
            img_path = path+'/'+folder+'/'+img_name
            img = cv2.imread(img_path)
            res = img2HOG(img)
            hog = list(res["HOG"])
            hog.append(class_brand.index(folder))
            response.append(hog)
    
    return response


In [ ]:
# %% Read Data Train
path = r'Cars Dataset\train'
datasetHOG = readData(path)

In [ ]:
# %% Save data train into pickle
write_path = "imgHOG.pkl"
pickle.dump(datasetHOG,open(write_path,"wb"))


In [6]:
# %% Load data train
imgHog = pickle.load(open('imgHog.pkl','rb'))
print(len(imgHog))

3352


In [7]:
# %% Convert list to array
imgHog_np = np.array(imgHog)
x_train = imgHog_np[:,0:-1]
y_train = imgHog_np[:,-1]

In [ ]:
# %% check size data
print(x_train.shape)
print(y_train.shape)

In [ ]:
# %% Read data Test 
path_test = r'Cars Dataset\test'
dataset_test = readData(path_test)

In [ ]:
# %% save data Test
write_path = "imgHOG_test.pkl"
pickle.dump(dataset_test,open(write_path,"wb"))


In [8]:
# %% Load data Test
dataset_test = pickle.load(open('imgHOG_test.pkl','rb'))
print(len(dataset_test))

813


In [9]:
# %% Convert list to array
dataset_test_np = np.array(dataset_test)
x_test = dataset_test_np[:,0:-1]
y_test = dataset_test_np[:,-1]

In [ ]:
# %% Train Model DecisionTree
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)


In [11]:
# %% Predict model DecisionTree
from sklearn import metrics

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test,y_pred)
cofus_mect = metrics.confusion_matrix(y_test,y_pred)
print("accuracy = ",acc*100)
print('Confusion Mectrix :\n',cofus_mect)


accuracy =  48.33948339483395
Confusion Mectrix :
 [[ 78  17  19  17  18  21  29]
 [ 11  40   2   1   2   2   9]
 [ 11   2  26   7   6   8  15]
 [ 24   4   9  13   4   6  14]
 [ 12  10   6   3  50   1  20]
 [ 13   2   4   7   5  66   9]
 [ 24   6   6  11  16   7 120]]


In [ ]:
# %% Save model DecisionTree
path_model = 'cls_carModel.pkl'
pickle.dump(clf,open(path_model,'wb'))


In [ ]:
# %% Train Model Xgboost
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(x_train,y_train)


In [13]:
# %% Predict model Xgboost
y_pred_xgb = xgb.predict(x_test)
acc_xgb = metrics.accuracy_score(y_test,y_pred_xgb)
cofus_mect_xgb = metrics.confusion_matrix(y_test,y_pred_xgb)
print("accuracy = ",acc_xgb*100)
print('Confusion Mectrix :\n',cofus_mect_xgb)


accuracy =  65.68265682656826
Confusion Mectrix :
 [[162   3   2   5   5   8  14]
 [  7  41   2   2   3   1  11]
 [  9   0  37   0   5   3  21]
 [ 44   2   3  13   3   1   8]
 [ 14   0   2   2  51   3  30]
 [ 15   1   1   0   3  78   8]
 [ 25   0   1   4   4   4 152]]


In [ ]:
# %% Save model Xgboost
path_model = 'model_XGB.pkl'
pickle.dump(xgb,open(path_model,'wb'))


In [ ]:
# %%  Train Model SVC
from sklearn.svm import SVC

svc = SVC(kernel='rbf', probability=True)
svc.fit(x_train,y_train)


In [16]:
# %% Predict model SVC
y_pred_svc = svc.predict(x_test)
acc_svc = metrics.accuracy_score(y_test,y_pred_svc)
cofus_mect_svc = metrics.confusion_matrix(y_test,y_pred_svc)
print("accuracy = ",acc_svc*100)
print('Confusion Mectrix :\n',cofus_mect_svc)

accuracy =  65.43665436654366
Confusion Mectrix :
 [[173   0   1   3   4   3  15]
 [ 14  32   2   0   1   0  18]
 [  9   0  31   0   3   1  31]
 [ 48   0   3  10   1   0  12]
 [ 19   0   2   1  52   1  27]
 [ 16   0   2   0   3  73  12]
 [ 24   1   0   0   3   1 161]]


In [ ]:
# %% Save model SVC
path_model = 'model_svc.pkl'
pickle.dump(svc,open(path_model,'wb'))


In [ ]:
# %% VotingClassifier
from sklearn.ensemble import VotingClassifier

vot =  VotingClassifier(estimators=[
        ('dt', clf), ('xgb', xgb), ('svc', svc)],
        voting='hard')
vot.fit(x_train,y_train)


In [18]:
# %% Predict model VotingClassifier
y_pred_vot = vot.predict(x_test)
acc_vot = metrics.accuracy_score(y_test,y_pred_vot)
cofus_mect_vot = metrics.confusion_matrix(y_test,y_pred_vot)
print("accuracy = ",acc_vot*100)
print('Confusion Mectrix :\n',cofus_mect_vot)


accuracy =  66.29766297662977
Confusion Mectrix :
 [[181   0   0   2   2   3  11]
 [ 14  38   2   1   1   0  11]
 [ 17   0  34   1   2   1  20]
 [ 51   3   3  11   0   0   6]
 [ 19   1   2   1  51   1  27]
 [ 17   0   1   2   5  74   7]
 [ 30   0   0   4   5   1 150]]


In [ ]:
# %% Save model vot
path_model = 'model_vot.pkl'
pickle.dump(vot,open(path_model,'wb'))
